In [28]:
import os
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

from sklearn import preprocessing, pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from cfmining.mip_algorithms import LinearRecourseActions, LinearRecourseActionsMulti
from cfmining.algorithms import MAPOCAM, BruteForce, Greedy
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import MonotoneClassifier, GeneralClassifier, TreeClassifier, LinearClassifier, LinearRule
from cfmining.action_set import ActionSet

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

## Preparing dataset

In [3]:
giveme_df = pd.read_csv('data/student/student-mat.csv', sep=';', index_col=None).dropna()
y = 1*(giveme_df['G3']>=10)
X = giveme_df.drop(['G1', 'G2', 'G3'], axis=1)

In [4]:
X = giveme_df.drop(['G1', 'G2', 'G3'], axis=1)
X['school'] = X['school'].map({'GP': 0, 'MS': 1})
X['sex'] = X['sex'].map({'M': 0, 'F': 1})
X['address'] = X['address'].map({'R': 0, 'U': 1})
X['famsize'] = X['famsize'].map({'LE3': 0, 'GT3': 1})
X['Pstatus'] = X['Pstatus'].map({'A': 0, 'T': 1})
X['reason'] = X['reason'].map({'other':0, 'home':1, 'reputation':2, 'course':3})
X['Mjob'] = X['Mjob'].map({'other':0, 'health':1, 'services':2, 'at_home':3, 'teacher':4})
X['Fjob'] = X['Fjob'].map({'other':0, 'health':1, 'services':2, 'at_home':3, 'teacher':4})
X['guardian'] = X['guardian'].map({'other':0, 'mother':1, 'father':2})
X['schoolsup'] = X['schoolsup'].map({'no': 0, 'yes': 1})
X['famsup'] = X['famsup'].map({'no': 0, 'yes': 1})
X['paid'] = X['paid'].map({'no': 0, 'yes': 1})
X['activities'] = X['activities'].map({'no': 0, 'yes': 1})
X['nursery'] = X['nursery'].map({'no': 0, 'yes': 1})
X['higher'] = X['higher'].map({'no': 0, 'yes': 1})
X['internet'] = X['internet'].map({'no': 0, 'yes': 1})
X['romantic'] = X['romantic'].map({'no': 0, 'yes': 1})

In [5]:
Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=100)

## Setting ActionSet base parameters

In [6]:
action_set_base = ActionSet(X = X)
for feat in action_set_base:
    if feat.name in ['school', 'sex', 'age']:
        feat.mutable = False
        #feat.flip_direction = 1
        feat.step_direction = -1
    if feat.name in ['Mjob', 'Fjob']:
        feat.mutable = False
        #feat.flip_direction = 1
        feat.step_direction = 1
    if feat.name in ['famsize', 'traveltime', 'failures',
                    'romantic', 'Dalc', 'Walc', 'absences',
                     'goout', 'Pstatus', 'reason', 'activities',
                    'schoolsup',]:
        feat.mutable = True
        #feat.flip_direction = -1
        feat.step_direction = -1
    if feat.name in ['address', 'Medu',
                     'Fedu', 'guardian',
                    'studytime', 'schoolsup', 'famsup',
                     'paid', 'nursery',
                     'higher', 'internet', 'famrel',
                    'freetime', 'health']:
        feat.mutable = True
        #feat.flip_direction = -1
        feat.step_direction = 1
    feat.step_type = 'absolute'
    if feat.name in ['absences']:
        feat.step_type = 'relative'
        feat.step_size = 0.1
    feat.step_size = 1
    feat.update_grid()

## Logistic regression model

In [7]:
stand = preprocessing.StandardScaler()
clf_base = LogisticRegression(max_iter=1000, class_weight='balanced')
clf = pipeline.Pipeline([('std', stand), ('clf', clf_base)])

grid = GridSearchCV(
  clf, param_grid={'clf__C': np.logspace(-4, 3)},
  cv=5,
  scoring='roc_auc',
  verbose=0
)

grid.fit(Xtr, ytr)
clf_lgr = grid.best_estimator_

In [8]:
thresh = 0.5
coefficients = clf_lgr['clf'].coef_[0]/clf_lgr['std'].scale_
intercept = clf_lgr['clf'].intercept_[0]-coefficients@clf_lgr['std'].mean_
clf_lgr_mapocam = MonotoneClassifier(clf_lgr, X=Xtr, y=ytr, threshold=thresh)
#clf_lgr_mapocam = LinearRule(coefficients, Xtr, threshold=-intercept+(np.log(thresh / (1. - thresh))))

/home/marcos/Documents/Experiments/envs/mapocam/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/marcos/Documents/Experiments/envs/mapocam/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


## Preparing counterfactual parameters

In [9]:
action_set = copy.deepcopy(action_set_base)

print('ActionSet stats')
print('Number of actionable features:', sum([action.actionable for action in action_set]))
print('Mean number of actions per feature:', np.nanmean([len(action._grid) if action.actionable else np.nan for action in action_set]))
print('Max number of actions per feature:', np.nanmax([len(action._grid) if action.actionable else np.nan for action in action_set]))

ActionSet stats
Number of actionable features: 25
Mean number of actions per feature: 3.24
Max number of actions per feature: 5.0


In [10]:
scores = pd.Series(clf_lgr.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < thresh].index

In [11]:
percCalc = PercentileCalculator(action_set=action_set)

## Linear programming

In [12]:
lp_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActions(action_set, individual, coefficients, intercept, thresh)
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_lgr_results[i] = {}
    lp_lgr_results[i]['enum'] = lp
    lp_lgr_results[i]['indiv'] = individual
    lp_lgr_results[i]['solution'] = [int(s) if act.variable_type=='int' else float(s) for s, act in zip(lp.solution, action_set)]
    lp_lgr_results[i]['stats'] = lp.stats
    lp_lgr_results[i]['recourse'] = lp.recourse
    lp_lgr_results[i]['time'] = lp_time
    lp_lgr_results[i]['cost'] = criteria.f(lp_lgr_results[i]['solution']) if ~any(np.isnan(lp_lgr_results[i]['solution'])) else float('inf')

100%|██████████| 35/35 [00:00<00:00, 40.50it/s]


## Greedy

In [13]:
greedy_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    greedy = Greedy(action_set, individual, clf_lgr_mapocam, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_lgr_results[i] = {}
    greedy_lgr_results[i]['solution'] = greedy.solution
    greedy_lgr_results[i]['time'] = greedy_time
    greedy_lgr_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

100%|██████████| 35/35 [00:00<00:00, 56.16it/s]


## MAPOCAM

In [14]:
mapocam_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_lgr_results[i] = {}
    mapocam_lgr_results[i]['enum'] = mapocam
    mapocam_lgr_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_lgr_results[i]['time'] = mapocam_time
    mapocam_lgr_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 35/35 [00:00<00:00, 40.97it/s]


## Performance comparison for single objective


In [36]:
import sklearn, yaml

ModuleNotFoundError: No module named 'ruamel'

In [16]:
lp_lgr_costs = np.array([lp_lgr_results[i]['cost'] for i in denied_individuals])
greedy_lgr_costs = np.array([greedy_lgr_results[i]['cost'] for i in denied_individuals])
mapocam_lgr_costs = np.array([mapocam_lgr_results[i]['cost'] for i in denied_individuals])

lp_lgr_time = np.array([lp_lgr_results[i]['time'] for i in denied_individuals])
greedy_lgr_time = np.array([greedy_lgr_results[i]['time'] for i in denied_individuals])
mapocam_lgr_time = np.array([mapocam_lgr_results[i]['time'] for i in denied_individuals])

In [49]:
lgr_dict = {'Time performance':
            {
                'mi-logistic': float(np.mean(lp_lgr_time)),
                'greedy': float(np.mean(greedy_lgr_time)),
                'MAPOCAM': float(np.mean(mapocam_lgr_time))
            },
            'Best solution rate':
           {
               'mi-logistic': float(np.mean(lp_lgr_costs==best_costs)),
               'greedy': float(np.mean(greedy_lgr_costs==best_costs)),
               'MAPOCAM':float(np.mean(mapocam_lgr_costs==best_costs))
           }
    
}
save_result(lgr_dict, )

## Comparing Pareto enumeration

## Percentile vs changes

In [19]:
clf_lgr_mapocam = MonotoneClassifier(clf_lgr, X=Xtr, y=ytr, threshold=thresh)
#clf_lgr_mapocam = LinearRule(coefficients, Xtr, threshold=-intercept+(np.log(thresh / (1. - thresh))))

In [20]:
#%%prun
mapocam_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_pc[i] = {}
    mapocam_results_pc[i]['enum'] = mapocam
    mapocam_results_pc[i]['time'] = mapocam_time
    mapocam_results_pc[i]['num'] = len(mapocam.solutions)

100%|██████████| 35/35 [00:13<00:00,  2.66it/s]


In [21]:
'''
%%prun
mapocam_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_pc[i] = {}
    mapocam_results_pc[i]['enum'] = mapocam
    mapocam_results_pc[i]['time'] = mapocam_time
    mapocam_results_pc[i]['num'] = len(mapocam.solutions)
'''

"\n%%prun\nmapocam_results_pc = {}\nfor i in tqdm(denied_individuals):\n    individual = Xts.iloc[i].values\n    \n    start = time.perf_counter()\n    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc), recursive=True)\n    mapocam.fit()\n    mapocam_time = time.perf_counter()-start\n    \n    mapocam_results_pc[i] = {}\n    mapocam_results_pc[i]['enum'] = mapocam\n    mapocam_results_pc[i]['time'] = mapocam_time\n    mapocam_results_pc[i]['num'] = len(mapocam.solutions)\n"

In [22]:
lp_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActionsMulti(action_set, individual, coefficients, intercept,
                                    thresh, max_changes=3, enumeration_type='remove_number_actions',
                                    compare=PercentileChangesCriterion(individual, percCalc))
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_results_pc[i] = {}
    lp_results_pc[i]['enum'] = lp
    lp_results_pc[i]['time'] = lp_time
    lp_results_pc[i]['num'] = len(lp.solutions)

100%|██████████| 35/35 [00:00<00:00, 56.31it/s]


## Features

In [23]:
mapocam_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_feat[i] = {}
    mapocam_results_feat[i]['enum'] = mapocam
    mapocam_results_feat[i]['time'] = mapocam_time
    mapocam_results_feat[i]['num'] = len(mapocam.solutions)

100%|██████████| 35/35 [00:08<00:00,  4.22it/s]


In [24]:
lp_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActionsMulti(action_set, individual, coefficients, intercept,
                                    thresh, max_changes=3, enumeration_type='remove_dominated')
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_results_feat[i] = {}
    lp_results_feat[i]['enum'] = lp
    lp_results_feat[i]['time'] = lp_time
    lp_results_feat[i]['num'] = len(lp.solutions)

100%|██████████| 35/35 [00:47<00:00,  1.35s/it]


In [25]:
mapocam_sols_pc = np.array([mapocam_results_pc[i]['num'] for i in denied_individuals])
lp_sols_pc = np.array([lp_results_pc[i]['num'] for i in denied_individuals])

mapocam_sols_feat = np.array([mapocam_results_feat[i]['num'] for i in denied_individuals])
lp_sols_feat = np.array([lp_results_feat[i]['num'] for i in denied_individuals])

mapocam_time_pc = np.array([mapocam_results_pc[i]['time'] for i in denied_individuals])
lp_time_pc = np.array([lp_results_pc[i]['time'] for i in denied_individuals])

mapocam_time_feat = np.array([mapocam_results_feat[i]['time'] for i in denied_individuals])
lp_time_feat = np.array([lp_results_feat[i]['time'] for i in denied_individuals])

## Testing same number of solutions


In [26]:
print('Changes vs percentile:', np.mean(lp_sols_pc==mapocam_sols_pc)==1)
print('Features:', np.mean(lp_sols_feat==mapocam_sols_feat)==1)

Changes vs percentile: True
Features: True


In [27]:
print('Changes vs percentile:')
print('Ustun:',np.mean(lp_time_pc))
print('Proposed:', np.mean(mapocam_time_pc))
print('Features:')
print('Ustun:',np.mean(lp_time_feat))
print('Proposed:', np.mean(mapocam_time_feat))

Changes vs percentile:
Ustun: 0.017469283203328294
Proposed: 0.3758050606319947
Features:
Ustun: 1.3538219116811108
Proposed: 0.236417396624373
